## LHC papers
Resources:
* https://github.com/kamyu104/LeetCode-Solutions/blob/master/Python/h-index.py
* https://www.analyticsvidhya.com/blog/2017/09/building-machine-learning-model-fun-using-orange/
* https://orange.biolab.si/blog/2016/07/18/network-analysis-with-orange/

In [1]:
import pandas as pd
import time
import random

## Functions

In [2]:
def normalize(df,nd={'title':'title_TeX',
                     'doi':'DOI'}):
    for k in nd.keys():
        if k in df:
            df[nd[k]]=df[k].apply(
                    lambda d: d[0] if isinstance(d,list) else d
                )
            #special cases
            if k=='title':
                df[nd[k]]=df[nd[k]].apply(
                    lambda d: d.get(k) if isinstance(d,dict) else d)
    return df

In [3]:
lhc_query='cn cms or cn atlas or cn lhcb or cn alice'

## Get query results

In [4]:
q='{} and collection:published'.format(lhc_query)
q=q.replace(' ','+')
#df=pd.read_json(
#    'https://inspirehep.net/search?p=find+{}&rg=250&of=recjson'.format(q))

In [164]:
from urllib.error import HTTPError
def to_json_append(df,file):
    '''
    Load the file with
    pd.read_json(file,orient='records',lines=True)
    '''
    df.to_json('tmp.json',orient='records',lines=True)
    #append
    f=open('tmp.json','r')
    k=f.read()
    f.close()
    f=open(file,'a')
    f.write('\n') #Prepare next data entry
    f.write(k)
    f.close()
    
def get_query(q):
    cc=pd.DataFrame()
    jrec=cc.shape[0]
    while jrec%250==0:
        query='http://inspirehep.net/search?ln=en&p={}&rg=250&jrec={}&of=recjson'.format(
              q,jrec+1)
        try:
            c=pd.read_json(query)
        except HTTPError:
            print('WARNING → HTTPError: HTTP Error 404: Not Found')
            c=pd.DataFrame()
        c=normalize(c)
        if not c.empty:
            cc=(cc.append(c,sort=True)).reset_index(drop=True)
            jrec=cc.shape[0]
        else:
            jrec=jrec+1 # To exit the loop
        print('jrec',jrec)
        time.sleep(1)
    return cc

def get_citations(recid,sleep=3):
    cc=pd.DataFrame()
    jrec=cc.shape[0]
    while jrec%250==0:
        qcites='http://inspirehep.net/search?ln=en&p=refersto:recid:{}&rg=250&jrec={}&of=recjson'.format(
              recid,jrec+1)
        try:
            c=pd.read_json(qcites)
        except HTTPError:
            print('WARNING → HTTPError: HTTP Error 404: Not Found')
            c=pd.DataFrame()
        c=normalize(c)
        for k in ['recid','authors','DOI','publication_info']:
            if k not in c:
                c[k]=None

        if not c.empty:
            c['cites']=c.apply(lambda row: 
                               {'recid':row['recid'],
                                'authors':row['authors'],
                                'publication_info':row['publication_info'],
                                'DOI'    :row['DOI']
                               },   axis=1)
        if not c.empty:
            cc=(cc.append(c,sort=True)).reset_index(drop=True)
            #Temporal storage
            if 'cites' in cc:
                try:
                    tmpdf=cc.drop('cites',axis='columns')
                    #Be sure to start with empty 'tmp_cites.json'
                    to_json_append(tmpdf,'tmp_cites.json')
                except:
                    pass

            jrec=cc.shape[0]
        else:
            jrec=jrec+1 # To exit the loop
        time.sleep(sleep)
    return cc
def get_citations_list(recid):
    c=get_citations(recid)
    print(recid,c.shape[0])
    time.sleep(random.randint(1,5))
    if not c.empty:
        c[['cites']]
        return list(c['cites'].values)
    else:
        return None

In [6]:
RELOAD=False
if RELOAD:
    lhc=get_query(q)

In [7]:
if RELOAD:
    lhc.to_json('lhc.json.gz',compression='gzip')

In [8]:
lhc=pd.read_json('lhc.json.gz',compression='gzip')

In [9]:
lhc.shape

(3584, 45)

In [56]:
df=lhc.copy()

## Add citations metadata

Add citations column

In [69]:
df=df.sort_values('number_of_citations').reset_index(drop=True)

In [70]:
dff=df[df['number_of_citations']==0].reset_index(drop=True)
dfc=df[df['number_of_citations'] >0].reset_index(drop=True)

In [140]:
nstr=2500;nend=3000
qq=dfc[nstr:nend].reset_index(drop=True)

In [141]:
s=time.time()
#df['cites']=df['recid'].apply(get_citations_list)
qq['cites']=qq['recid'].apply(get_citations_list)
tot=time.time()-s
print(tot)

1188683
1388185
1185104
1282445
1264662
790370
1289223
1300821
1487277
912706
1092979
1341609
1364361
1380192
1373300
1342447
1081581
1398582
1084331
1465513
1085982
834354
1118162
1676214
1487472
892704
921788
1374219
1251905
1426515
1188009
1517191
1323630
1489183
1296259
1624165
1373914
1298807
1091070
1297226
1191022
895742
1295687
1355375
1298812
1644900
1247976
1180197
1309874
1399051
1644899
1362177
857644
1091527
1224805
1298030
1223730
1219000
1127505
1126148
1112562
1189823
921468
1462258
1233089
922291
1114753
1315947
1097057
1591328
1208913
1353390
924458
1431986
889546
1120736
1243156
1477585
1228506
1223327
891834
1318483
939559
1197323
1446983
1184952
1639856
1641262
1357991
1325553
1222336
1126963
1391147
1469453
917526
1126962
1653948
1478792
1184938
1488580
1224539
1472319
1308737
1296831
1238809
915920
1082456
896211
913576
1111995
1269437
856566
871487
860477
1121877
1630632
1210881
1382594
1090895
895581
1335501
917599
1488280
1282447
1104744
1114034
1125576
896764

In [142]:
kk=qq.copy()
kk['nc']=kk['cites'].apply(lambda l: len(l) if isinstance(l,list) else 0)
dfy=kk[kk.apply(lambda row: row['number_of_citations']<=row['nc'],axis=1)].reset_index(drop=True)
dfn=kk[kk.apply(lambda row: row['number_of_citations']>row['nc'],axis=1)].reset_index(drop=True)
dfn.shape[0],dfn.shape[0]+dfy.shape[0],df.shape[0]

(500, 500, 3584)

In [138]:
FIX=True
if FIX:
    dff=(dff.append(qq,sort=True)).reset_index(drop=True)

In [131]:
sh=dff.shape[0]
if dfn.shape[0]==0:
    dff=(dff.append(qq,sort=True)).reset_index(drop=True)
    shend=dff.shape[0]
    print(sh,shend)
elif dfn.shape[0]>0:
    dff=(dff.append(dfy,sort=True)).reset_index(drop=True)
    qq=dfn.copy()
    print('some failed:',qq.shape[0])

some failed: 6


In [135]:
qq[qq.apply(lambda row: row['number_of_citations']>row['nc'],axis=1)][['nc','number_of_citations']]

,nc,number_of_citations
0,68,69
1,0,71
2,0,72
3,0,73
4,76,77
5,0,80


In [139]:
dff.to_json('dff_preliminar.json.gz',compression='gzip')

Fix missing cites

In [39]:
kkk=dfn[0:2]
kkk['cites']=kkk['recid'].apply(get_citations_list)

1738300
1485702


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
kkk['nc']=kkk['cites'].apply(lambda l: len(l) if isinstance(l,list) else 0)
kkk[kkk.apply(lambda row: row['number_of_citations']==row['nc'],axis=1)].reset_index(drop=True)[['nc','number_of_citations']]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,nc,number_of_citations
0,2,2
1,112,112


In [ ]:
kk['nc']=kk['cites'].apply(lambda l: len(l) if isinstance(l,list) else 0)
dfy=kk[kk.apply(lambda row: row['number_of_citations']==row['nc'],axis=1)].reset_index(drop=True)

In [19]:
dfn['cites']=dfn['recid'].apply(get_citations_list)

1738300
1485702
1485701


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


1485699
1485571
1485353
1485195
1484162
1483165
1737724
1483164
1482943
1481810
1481187
1480862
1480365


KeyboardInterrupt: 

In [24]:
kk['nc']=kk['cites'].apply(lambda l: len(l) if isinstance(l,list) else 0)
kk[kk.apply(lambda row: row['number_of_citations']==row['nc'],axis=1)].reset_index(drop=True)

,DOI,FIXME_OAI,abbreviated_title,abstract,accelerator_experiment,agency_code,authors,base,cataloguer_info,cites,...,subject,system_control_number,system_number,thesaurus_terms,title,title_TeX,title_additional,url,version_id,nc


In [28]:
kk['cites'].loc[0]

{'DOI': '10.1140/epjc/s10052-019-7389-9',
 'authors': [{'INSPIRE_number': 'INSPIRE-00570928',
   'affiliation': 'Calcutta, VECC',
   'first_name': 'Shreyasi',
   'full_name': 'Acharya, Shreyasi',
   'last_name': 'Acharya'},
  {'INSPIRE_number': 'INSPIRE-00261412',
   'affiliation': 'Rez, Nucl. Phys. Inst.',
   'first_name': 'Dagmar',
   'full_name': 'Adamova, Dagmar',
   'last_name': 'Adamova'},
  {'INSPIRE_number': 'INSPIRE-00541291',
   'affiliation': 'Calcutta, VECC',
   'first_name': 'Souvik Priyam',
   'full_name': 'Adhya, Souvik Priyam',
   'last_name': 'Adhya'},
  {'INSPIRE_number': 'INSPIRE-00657848',
   'affiliation': 'Goethe U., Frankfurt, Inst. Inform.',
   'first_name': 'Alexander',
   'full_name': 'Adler, Alexander',
   'last_name': 'Adler'},
  {'INSPIRE_number': 'INSPIRE-00571154',
   'affiliation': 'Lund U.',
   'first_name': 'Jonatan',
   'full_name': 'Adolfsson, Jonatan',
   'last_name': 'Adolfsson'},
  {'INSPIRE_number': 'INSPIRE-00180711',
   'affiliation': 'Panjab U

In [29]:
kk['number_of_citations'].loc[0]

2

In [22]:
kk=dfn.copy()

In [23]:
kk.shape

(3292, 46)

In [20]:
dfn['cites']

0       {'publication_info': {'reference': 'Eur. Phys....
1       {'publication_info': {'pagination': '147-170',...
2       {'publication_info': {'pagination': '055', 'vo...
3       {'publication_info': {'pagination': 'P01020', ...
4       {'publication_info': [{'pagination': '021801',...
5       {'publication_info': {'pagination': '40', 'vol...
6       {'publication_info': {'pagination': '435-456',...
7       {'publication_info': {'pagination': '92', 'vol...
8       {'publication_info': [{'pagination': 'T10004',...
9       {'publication_info': {'reference': 'Phys. Lett...
10      {'publication_info': {'reference': 'Phys. Lett...
11      {'publication_info': {'pagination': '261801', ...
12      {'publication_info': {'pagination': '253-262',...
13      {'publication_info': {'pagination': '653', 'vo...
14      {'publication_info': {'pagination': '123', 'vo...
15      {'publication_info': {'reference': 'Phys. Lett...
16      {'publication_info': {'pagination': 'T10005', ...
17      {'publ

In [16]:
df=(dfy.append(dfn)).reset_index(drop=True)

## Filter published papers (query already filtered!)

In [72]:
dfp=df[~df['publication_info'].isna()].reset_index(drop=True)

Get the Journal info from crossref api

In [73]:
journal={}

In [74]:
import time
def get_journal_info(row,journal=journal,abrv_journal='publication_info'
                                        ,abrv_journal_title='title'
                                        ,doi='doi'
                                        ,container_title='container_title'
                                        ,issn='ISSN'):
    
    try:
        j=row.get(abrv_journal).get(abrv_journal_title)
    except:
        j=None
    if j and not journal.get(j):
        journal.update( {j:{}} )
    
    if not journal.get(j) and row.get(doi):
        strdoi=row.get(doi)
        try:
            di=pd.read_json( 'https://api.crossref.org/works/{}'.format( strdoi  ) )
        except:
            di=pd.DataFrame()
        if j in journal and not journal.get(j).get(container_title):
            try:
                ct=di.loc['container-title'][0][0]
                journal[j].update({container_title:ct})

            except:
                pass
        if j in journal and not journal.get(j).get(issn):
            try:
                gis=di.loc['ISSN'].get('message')[0]
                journal[j].update({issn:gis})
            except:
                pass
        #Get a new journal crossref metadata here

        time.sleep(1)
        return di

In [75]:
di=dfp.apply(get_journal_info,axis=1)

In [76]:
journal

{'Eur.Phys.J.': {'ISSN': '1434-6044',
  'container_title': 'The European Physical Journal C'},
 'Europhys.Lett.': {'ISSN': '0295-5075',
  'container_title': 'Europhysics Letters (EPL)'},
 'Int.J.Mod.Phys.': {'ISSN': '0217-751X',
  'container_title': 'International Journal of Modern Physics A'},
 'J.Phys.': {'ISSN': '0954-3899',
  'container_title': 'Journal of Physics G: Nuclear and Particle Physics'},
 'JCAP': {'ISSN': '1475-7516',
  'container_title': 'Journal of Cosmology and Astroparticle Physics'},
 'JHEP': {'ISSN': '1029-8479',
  'container_title': 'Journal of High Energy Physics'},
 'JINST': {'ISSN': '1748-0221',
  'container_title': 'Journal of Instrumentation'},
 'Mod.Phys.Lett.': {'ISSN': '0217-7323',
  'container_title': 'Modern Physics Letters A'},
 'Nucl.Phys.': {'ISSN': '0550-3213', 'container_title': 'Nuclear Physics B'},
 'Phys.Lett.': {'ISSN': '0370-2693', 'container_title': 'Physics Letters B'},
 'Phys.Rept.': {'ISSN': '0370-1573', 'container_title': 'Physics Reports'

Normalize data

In [77]:
dfp=normalize(dfp)

Add Journal metadata

In [78]:
dfp['Year']=dfp['publication_info'].apply(
    lambda d: d.get('year') if isinstance(d,dict) else None).apply(
    lambda y: int(y) if isinstance(y,str) else 0)

dfp['Journal_name']=dfp['publication_info'].apply(lambda d: journal.get( 
                                        d.get('title') ).get('container_title') 
                                        if isinstance(d,dict) 
                                        else None)
dfp['ISSN']=dfp['publication_info'].apply(lambda d: journal.get( 
                                        d.get('title') ).get('ISSN') 
                                        if isinstance(d,dict) 
                                        else None)

Extract author metadata for affiliation UdeA

In [79]:
dfp['udea_authors']=dfp['authors'].apply(lambda l: [ #loop for each author in the list
        '{} {}'.format(d.get('first_name'), d.get('last_name')) #Extract full name 
         for d in l 
           if str(d.get('affiliation')).find('Antioquia U.')>-1 #For UdeA authors
                               ])
dfp['first_udea_author']=dfp['udea_authors'].str[0]

Prepare "Sostenibilidad" output

In [86]:
dfp.to_json('gfif.json.gz',compression='gzip')

In [87]:
kk=pd.read_json('gfif.json.gz',compression='gzip')

In [ ]:
sost=dfp[['title_TeX','Journal_name','first_udea_author','Year','ISSN','DOI']]

In [ ]:
sost[sost.Year>2017]#.to_excel('sost.xlsx',index=False)

In [81]:
%%bash
git commit -am 'lc'
git push origin master

[master ba8b47f] lc
 1 file changed, 348 insertions(+), 6 deletions(-)


To github.com:restrepo/inspire.git
   0448bba..ba8b47f  master -> master
